# Проект: Сервис для чтения книг по подписки.

Цель: сформулировать ценностное предложение для создания нового продукта.

Задачи: 
- проанализировать базу данных с книгами
- найдем вышедшие книги в текущем столетии, 
- посчитаем количество обзоров и среднюю оценку книг
- выделим из популярных авторов, автора с самыми высокими оценками
- выявим среднее количество обзоров от пользователей, которые активно их оставляют


In [37]:
# импортируем библиотеки
import pandas as pd
import sqlalchemy as sa
from sqlalchemy import text, create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})
# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [38]:
def get_sql_data(query:str, engine:sa.engine.base.Engine=engine) -> pd.DataFrame:
    with engine.connect() as con:
        return pd.read_sql(sql=sa.text(query), con=con)

In [39]:
def sql_table_info(table):
    query = '''
               SELECT * FROM {}'''.format(table)

In [40]:
query = '''SELECT * FROM authors LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [41]:
query = '''SELECT * FROM publishers LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [42]:
query = '''SELECT * FROM ratings LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [43]:
query = '''SELECT * FROM reviews LIMIT 5'''
con=engine.connect()
pd.io.sql.read_sql(sql=text(query), con = con)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [44]:
query = '''SELECT COUNT(review_id)
FROM reviews;'''
get_sql_data(query)

,count
0,2793


Сколько же книг вышло после 1 января 2000 года.


In [45]:
query = '''SELECT COUNT(DISTINCT book_id)
FROM books
WHERE publication_date >= '2000-01-01';'''
get_sql_data(query)

,count
0,821


Этот параметр позволит разработчикам составить представление сколько всего книг вышло в этом веке, как быстро появляются и издаются новые книги. 

Для каждой книги посчитаем количество обзоров и среднюю оценку.


In [46]:
query = '''WITH a AS (SELECT  b.title,
                  COUNT(r.review_id) count_review,
                  AVG(a.rating)
           FROM books AS b LEFT JOIN reviews AS r ON b.book_id = r.book_id
           LEFT JOIN ratings AS a ON b.book_id = a.book_id
           GROUP BY b.book_id
           ORDER BY COUNT(r.review_id) DESC)
           SELECT SUM(count_review)
           FROM a
'''
get_sql_data(query)

,sum
0,26167.0


In [47]:
query = '''WITH a AS (SELECT  b.title,
                  COUNT(DISTINCT r.review_id) count_review,
                  AVG(a.rating)
           FROM books AS b LEFT JOIN reviews AS r ON b.book_id = r.book_id
           LEFT JOIN ratings AS a ON b.book_id = a.book_id
           GROUP BY b.book_id
           ORDER BY COUNT(r.review_id) DESC)
           SELECT SUM(count_review)
           FROM a
'''
get_sql_data(query)

,sum
0,2793.0


In [48]:
query = '''SELECT  b.title,
                  COUNT(DISTINCT r.review_id) count_review,
                  AVG(a.rating)
           FROM books AS b LEFT JOIN reviews AS r ON b.book_id = r.book_id
           LEFT JOIN ratings AS a ON b.book_id = a.book_id
           GROUP BY b.book_id
           ORDER BY COUNT(r.review_id) DESC
           LIMIT 10;
'''
get_sql_data(query)

,title,count_review,avg
0,Twilight (Twilight #1),7,3.662500
1,The Hobbit or There and Back Again,6,4.125000
2,The Catcher in the Rye,6,3.825581
3,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
5,Angels & Demons (Robert Langdon #1),5,3.678571
6,Harry Potter and the Order of the Phoenix (Har...,5,4.186667
7,The Lightning Thief (Percy Jackson and the Oly...,6,4.080645
8,Animal Farm,5,3.729730
9,The Fellowship of the Ring (The Lord of the Ri...,5,4.391892


Видно, что есть у самых популярных книг, не всегда высокие оценки. Книга может вызывать среди читателей неоднозначное мнение, что может способствовать ее горячему обсуждению и, следовательно, популярности произведения. 

Издательство, которое выпустило наибольшее число книг толще 50 страниц (исключим брошюры из анализа).


In [49]:
query = '''SELECT p.publisher, 
                  COUNT(b.book_id)
           FROM books AS b LEFT JOIN publishers AS p ON b.publisher_id = p.publisher_id
           WHERE b.num_pages > 50
           GROUP BY p.publisher
           ORDER BY COUNT(b.book_id) DESC
           LIMIT 1;
'''
get_sql_data(query)

,publisher,count
0,Penguin Books,42


Это издательство активнее других печатает новые произведения авторов. Оно выпустило 42 новых книги. 

Автор с самой высокой средней оценкой книг(будет учитывать книги с 50 и более оценками)

In [50]:
query = '''WITH a AS (SELECT book_id, 
                             AVG(rating) AS rating
                      FROM ratings
                      GROUP BY book_id
                      HAVING COUNT(rating_id) > 50
                      ORDER BY AVG(rating))
        SELECT au.author,
               AVG(a.rating)
        FROM books AS b INNER JOIN a ON b.book_id=a.book_id
        LEFT JOIN authors AS au ON b.author_id = au.author_id
        
        GROUP BY au.author
        ORDER BY AVG(a.rating) DESC
        LIMIT 1;
'''
get_sql_data(query)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.283844


Предпочтениями публики в 21 веке пользуется писательница Дж. Роулинг. Ее книги набрали самые высокие оценки среди пользователей.

Среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [51]:
query = '''WITH a AS 
          (SELECT username, 
                  COUNT(review_id) AS user_count
           FROM reviews
           WHERE username IN (SELECT username
                              FROM ratings 
                              GROUP BY username
                              HAVING COUNT(rating) > 48)
          GROUP BY username)
          SELECT AVG(user_count)
          FROM a;
'''
get_sql_data(query)

,avg
0,24.0


Активные читатели в среднем оставили 24 обзора на произведения.

Выводы:


    Мы получили цифры, на которые могут ориентироваться разработчики. Насколько много нужно закладывать серверов для продажи подписки с сервисами. Сколько нужно закладывать под отзывы на произведения и тд. 
    Очень популярные произведения не всегда обладают высокими оценками - это специфика книжного рынка.
    Самым понравившимся писателем на сегодня является Дж. Роулинг.
    Издательство, которое чаще всего издает новые произведения - Penguin Books. На это издательство можно ориентироваться либо за ним следить, чтобы сервис не теряя конкурентного преимущества и тоже покупал и публиковал новинки. 